In [1]:
!git clone https://github.com/DGU-OpenSW-Team6/ai.git
%cd ai

Cloning into 'ai'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 40 (delta 12), reused 22 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 25.37 KiB | 6.34 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/ai


In [2]:
!grep -v "python==" requirements.txt | pip install -r /dev/stdin

INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.4/755.4 MB 924.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.7/232.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 

In [3]:
from google.colab import files
files.upload()  # kaggle.json 업로드
!mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d vinothpandian/uisketch -p data/archive
!unzip -q data/archive/uisketch.zip -d data/archive

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/vinothpandian/uisketch
License(s): CC-BY-NC-SA-4.0
  0% 0.00/117M [00:00<?, ?B/s]
100% 117M/117M [00:00<00:00, 1.54GB/s]


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path

# ==============================
# 1️⃣ Dataset 클래스 정의
# ==============================
class SketchElementDataset(Dataset):
    def __init__(self, df, root_path, invert=False, img_size=224):
        self.df = df.reset_index(drop=True)
        self.root_path = Path(root_path)
        self.invert = invert
        self.img_size = img_size

        # 라벨을 숫자로 매핑
        self.classes = sorted(self.df['label'].unique())
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

        # ImageNet 정규화 상수
        self.IMAGENET_MEAN = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        self.IMAGENET_STD = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.root_path / row['name']
        label_str = row['label']
        y = self.class_to_idx[label_str]

        # 이미지 로드
        img = Image.open(img_path).convert("RGB")

        # (옵션) 반전
        if self.invert:
            img = ImageOps.invert(img)

        # 리사이즈
        img = img.resize((self.img_size, self.img_size))

        # numpy 변환 + [0,1] 스케일링
        arr = np.array(img).astype('float32') / 255.0
        arr = np.transpose(arr, (2, 0, 1))
        x = torch.tensor(arr, dtype=torch.float32)

        # ImageNet 정규화
        x = (x - self.IMAGENET_MEAN) / self.IMAGENET_STD

        y = torch.tensor(y, dtype=torch.long)
        return x, y


# ==============================
# 2️⃣ 데이터 로드 및 분할
# ==============================
SEED = 42
csv_path = "data/archive/labels.csv"     # ✅ labels.csv 경로
root_path = "data/archive"

train_df = pd.read_csv(csv_path)

inner_train_df, valid_df = train_test_split(
    train_df,
    test_size=0.1,
    random_state=SEED,
    stratify=train_df['label']
)

print(f"Train: {len(inner_train_df)}, Valid: {len(valid_df)}")

# ==============================
# 3️⃣ Dataset / DataLoader 생성
# ==============================
train_dataset = SketchElementDataset(inner_train_df, root_path, invert=False)
valid_dataset = SketchElementDataset(valid_df, root_path, invert=False)

train_loader = DataLoader(train_dataset, batch_size=48, shuffle=True,  num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=48, shuffle=False, num_workers=0, pin_memory=True)

print("✅ DataLoaders ready!")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-package

Train: 17100, Valid: 1900
✅ DataLoaders ready!


In [5]:
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet34, ResNet34_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(train_dataset.classes)

# 1) 모델 준비 (resnet34)
weights = ResNet34_Weights.IMAGENET1K_V1
model = resnet34(weights=weights)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# 2) 손실/옵티마
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

# 3) 배치 하나만 과적합 테스트
xb, yb = next(iter(train_loader))
xb_small, yb_small = xb.to(device), yb.to(device)

model.train()
for step in range(50):
    optimizer.zero_grad(set_to_none=True)
    logits = model(xb_small)
    loss = criterion(logits, yb_small)
    loss.backward()
    optimizer.step()

    if (step + 1) % 10 == 0:
        with torch.no_grad():
            preds = logits.argmax(1)
            acc = (preds == yb_small).float().mean().item()
        print(f"step {step+1:02d} | loss {loss.item():.4f} | acc {acc:.3f}")

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 110MB/s]


step 10 | loss 0.0105 | acc 1.000
step 20 | loss 0.0020 | acc 1.000
step 30 | loss 0.0010 | acc 1.000
step 40 | loss 0.0007 | acc 1.000
step 50 | loss 0.0005 | acc 1.000


In [6]:
from tqdm.auto import tqdm
from collections import Counter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(train_dataset.classes)

# 가벼운 모델
weights = ResNet34_Weights.IMAGENET1K_V1
model = resnet34(weights=weights)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# 불균형 가중치는 그대로
counts = Counter(inner_train_df['label'].tolist())
class_weights = torch.tensor([1.0 / counts[c] for c in train_dataset.classes],
                             dtype=torch.float32, device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

def run_epoch(loader, train=True):
    model.train(train)
    total_loss, correct, total = 0.0, 0, 0
    for xb, yb in tqdm(loader, leave=False):
        xb, yb = xb.to(device), yb.to(device)
        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            logits = model(xb)
            loss = criterion(logits, yb)
        if train:
            optimizer.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        total_loss += loss.item() * xb.size(0)
        correct += (logits.argmax(1) == yb).sum().item()
        total += xb.size(0)
    return total_loss/total, correct/total

EPOCHS = 3
best_val_acc = 0.0
for e in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_epoch(train_loader, True)
    va_loss, va_acc = run_epoch(valid_loader, False)
    print(f"[{e}/{EPOCHS}] train {tr_loss:.4f}/{tr_acc:.4f} | valid {va_loss:.4f}/{va_acc:.4f}")
    if va_acc > best_val_acc:
        best_val_acc = va_acc
        torch.save(model.state_dict(), "ui_classifier.pt")
        print("✅ saved")
print("best:", best_val_acc)

files.download("ui_classifier.pt")

  0%|          | 0/357 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

[1/3] train 0.8947/0.7292 | valid 0.6822/0.7911
✅ saved


  0%|          | 0/357 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/357 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

[3/3] train 0.4020/0.8693 | valid 0.5613/0.8268
✅ saved
best: 0.8268421052631579


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>